<a href="https://colab.research.google.com/github/joaquinhojman/ArbolesDeDecision/blob/main/ModeloLightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve


from sklearn.model_selection import train_test_split

In [2]:
url_train = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_train.csv'
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'
url_test_kaggle = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test_kaggle.csv'

data_train = pd.read_csv(url_train)
data_test = pd.read_csv(url_test)
data_test_kaggle = pd.read_csv(url_test_kaggle)

In [3]:
# Separamos los set de datos (data_train y data_test) de sus columnas 'Target'
# 'x' son los datos, 'y' es el target:
#   --> x_train son los datos con los que entrenar
#   --> y_train son las soluciones con las que comparar lo entrenado con x_train
#   --> x_test son los datos con los que probar
#   --> y_test son las soluciones con las que comparar lo probado con x_test
x_train = data_train.iloc[:, :-1]
y_train = data_train.iloc[:, -1]
x_test = data_test.iloc[:, :-1]
y_test = data_test.iloc[:, -1]

#Esto lo hizo chechu y esta bien, no cambiar.


In [4]:
import lightgbm as lgb

In [139]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'binary_logloss',              
    'num_leaves': 62,
    'learning_rate': 0.05,
    'feature_fraction': 0.22,
    'boosting': 'gbdt','min_samples_split':100
 } 

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)

preds = gbm.predict(x_test, num_iteration=gbm.best_iteration)

[1]	valid_0's binary_logloss: 0.658956
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.648688
[3]	valid_0's binary_logloss: 0.638566
[4]	valid_0's binary_logloss: 0.628957
[5]	valid_0's binary_logloss: 0.615469
[6]	valid_0's binary_logloss: 0.609813
[7]	valid_0's binary_logloss: 0.598988
[8]	valid_0's binary_logloss: 0.588421
[9]	valid_0's binary_logloss: 0.582722
[10]	valid_0's binary_logloss: 0.575231
[11]	valid_0's binary_logloss: 0.571289
[12]	valid_0's binary_logloss: 0.569792
[13]	valid_0's binary_logloss: 0.568237
[14]	valid_0's binary_logloss: 0.56513
[15]	valid_0's binary_logloss: 0.565999
[16]	valid_0's binary_logloss: 0.562327
[17]	valid_0's binary_logloss: 0.558412
[18]	valid_0's binary_logloss: 0.553659
[19]	valid_0's binary_logloss: 0.555097
[20]	valid_0's binary_logloss: 0.551887
[21]	valid_0's binary_logloss: 0.547743
[22]	valid_0's binary_logloss: 0.543628
[23]	valid_0's binary_logloss: 0.542537
[24]	valid_0's binary_loglo

In [141]:

error = log_loss(y_test, preds)
error

0.48116702322457333

In [7]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

In [142]:
# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = preds

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

In [143]:
# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.47376354355318984

In [144]:
# Sacamos el preds_test_agrupado_por_opp
preds_test_kaggle = gbm.predict(data_test_kaggle)

oportunidades_test = data_test_kaggle['Opportunity_ID']
oportunidades_test = oportunidades_test.to_frame()

oportunidades_test['Target'] = preds_test_kaggle

preds_test_kaggle_agrupado_por_opp = oportunidades_test.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_kaggle_agrupado_por_opp = preds_test_kaggle_agrupado_por_opp.reset_index()

preds_test_kaggle_agrupado_por_opp

,Opportunity_ID,Target
0,10689,0.451397
1,10690,0.326100
2,10691,0.068072
3,10692,0.323481
4,10693,0.598353
...,...,...
1562,12364,0.627461
1563,12365,0.153742
1564,12366,0.019273
1565,12367,0.101532


In [145]:
from google.colab import files

preds_test_kaggle_agrupado_por_opp.to_csv('preds_testlgbm.csv', index=False, header=True)
files.download('preds_testlgbm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>